In [ ]:
# Runtime is about 2.5 hours

In [ ]:
import numpy as np
import os
import torch
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader, TensorDataset, Dataset, Subset
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import transforms as T
from torchvision.transforms import ToTensor 
from utils.train import train_model,set_dataset, set_architecture

In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_data, test_data, class_num = set_dataset("MNIST")

In [ ]:
train_data_cifar, test_data_cifar, class_num_cifar = set_dataset("cifar10")

In [ ]:
batch_size = 128
train_loader_cifar = torch.utils.data.DataLoader(
                 dataset=train_data,
                 batch_size=batch_size,
                 shuffle=True)
test_loader_cifar = torch.utils.data.DataLoader(
                dataset=test_data,
                batch_size=batch_size,
                shuffle=True)

In [ ]:
target_model = set_architecture('Resnet18',device, 10)

In [ ]:
#setting train hyper-parameter
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(target_model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
if not os.path.exists("./models/attack_model/TLPA"):
    os.makedirs("./models/attack_model/TLPA")
for size in [10,30,50,70,90]:
    target_model = set_architecture('Resnet18',device, 10)
    target_model = target_model.to(device)
    train_len=len(train_data)

    sss = StratifiedShuffleSplit(n_splits=1, test_size=int(train_len*size/100), random_state=7)
    indices = list(range(len(train_data)))
    y_test= [y for _, y in train_data]

    for test_index, val_index in sss.split(indices, y_test):
        print(len(val_index), len(test_index))

    val_ds = Subset(train_data, val_index)
    test_ds = Subset(train_data, test_index)
    batch_size = 128
    train_loader = torch.utils.data.DataLoader(
                     dataset=val_ds,
                     batch_size=batch_size,
                     shuffle=True)
    test_loader = torch.utils.data.DataLoader(
                    dataset=test_data,
                    batch_size=batch_size,
                    shuffle=True)
   
    model, acc = train_model(target_model, criterion, optimizer_ft, exp_lr_scheduler,device, train_loader, test_loader, 10)
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 4:
            for param in child.parameters():
                param.requires_grad = False

    #setting train hyper-parameter
    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,device, train_loader_cifar, test_loader_ciafr, 10)
    torch.save(model,  './models/attack_model/TLPA/TLPA_'+str(size)+'.pt')
    acc_file=open('./models/attack_model/TLPA/TLPA_'+str(size)+'.txt','w')
    acc_file.write(str(acc))
    acc_file.close()